In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

pd.set_option('future.no_silent_downcasting', True)

## Definindo as colunas que serão usadas para treinar o modelo
team_map_results_columns_to_add  = ["Id", "Kills", "Deaths", "PlusMinus", "Adr", "Kast", "Rating"]
team_map_results_columns_to_drop = ["Id", "Kills", "Deaths", "Adr", "Kast"]

map_results = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in team_map_results_columns_to_add
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

## Definindo as colunas que serão retiradas para a análise
drop_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in team_map_results_columns_to_drop
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

### Listando as colunas que serão usadas das partidas, como: Jogadores, Mortes, Assistências etc.
matches_train_data_columns = ['eventId', 'matchId', 'mapBestOf'] + map_results

matches_results_train_data = pd.read_csv('../../data/raw/matches_results.csv')
matches_train_data = pd.read_csv('../../data/raw/matches.csv')

matches_train_data = matches_train_data[matches_train_data_columns]

matches_results_train_data_columns = ['eventId', 'matchId', 'TeamOneScore', 'TeamTwoScore', 'teamOneWon', 'teamTwoWon']
matches_results_train_data = matches_results_train_data[[col for col in matches_results_train_data.columns if any(s in col for s in matches_results_train_data_columns)]]

## Combinando os dados gerais das partidas com os detalhes das partidas
full_matches_train_data = pd.merge(matches_results_train_data, matches_train_data, on='matchId', how='inner')

## Data Wrangling
full_matches_train_data.fillna(0, inplace=True)
full_matches_train_data.replace("Not Available", 0, inplace=True)
full_matches_train_data.drop(['eventId_y', 'eventId_x', 'matchId'], axis=1, inplace=True)

full_matches_train_data.drop(drop_columns, axis=1, inplace=True)
full_matches_train_data.drop(['mapBestOf'], axis=1, inplace=True)

# Features (colunas que serão utilizadas para fazer a previsão)
X = full_matches_train_data.drop(['teamOneWon', 'teamTwoWon'], axis=1)
X = X.apply(pd.to_numeric, errors='coerce')

# Target (coluna que será prevista)
y = full_matches_train_data['teamOneWon']
y = y.apply(pd.to_numeric, errors='coerce')

print(f"Número de amostras em X?{X.shape} e y:{y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# Treinar o modelo com a base de treino
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Fazer previsões com o conjunto de teste
y_pred = model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do Modelo de Árvore de Decisão: {accuracy:.2f}')
print(f"Score do modelo de Árvore de Decisão: {model.score(X_test, y_test)}")

Número de amostras em X?(696, 300) e y:(696,)
Acurácia do Modelo de Árvore de Decisão: 0.73
Score do modelo de Árvore de Decisão: 0.7298850574712644


/var/folders/10/t4cd3lc515j88jjkvk8hvrbw0000gn/T/ipykernel_75312/3350044027.py:32: DtypeWarning: Columns (35,36,37,41,42,43,44,45,46,47,48,49,50,51,52,56,57,58,59,60,61,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,83,84,85,89,90,91,92,93,94,95,96,97,98,99,100,104,105,106,107,108,109,113,114,115,116,117,118,119,120,121,122,123,124,128,129,130,131,132,133,137,138,139,140,141,142,143,144,145,146,147,148,152,153,154,155,156,157,161,162,163,164,165,166,167,168,169,170,171,172,176,177,178,179,180,181,185,186,187,188,189,190,191,192,193,194,195,196,200,201,202,203,204,205,209,210,211,212,213,214,215,216,217,218,219,220,224,225,226,227,228,229,233,234,235,236,237,238,239,240,241,242,243,244,248,249,250,251,252,253,257,258,259,260,261,262,263,264,265,266,267,268,272,273,274) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_train_data = pd.read_csv('../../data/raw/matches.csv')
